In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'ch4': 3.6e-06}
band = [9]
atmpro = saw
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o ch4 band7
  atmpro = mls
  band = 7
  commitnumber = a06b618
  conc = 1.8e-06
  dv = 0.001
  klin = 2e-21
  molecule = ch4
  ng_adju = [0, 0]
  ng_refs = [2, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1380
  vmin = 1215
  w_diffuse = [(1.66, 1.66), (1.66, 1.66, 1.66)]
  wgt = [(0.7, 0.7), (0.4, 0.6, 0.75)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.122402                    1 -0.046598
0.000750        2 -0.114081                    2 -0.041361
0.001052        3 -0.095692                    3 -0.029444
0.001476        4 -0.075388                    4 -0.016397
0.002070        5 -0.053294                    5 -0.001977
0.002904        6 -0.029291                    6  0.013840
0.004074        7 -0.003300                    7  0.031102
0.005714        8  0.024754                    8  0.050097
0.008015        9  0.054917                    9  0.070824
0.011243       10  0.087211                   10  0.093415
0.015771       11  0.121671                   11  0.117996
0.022122       12  0.158230                   12  0.144689
0.031031       13  0.196616                   13  0.173480
0.043528       14  0.231138                   14  0.200713
0.061057       15  0.255888                   15  0.222741
0.085645       16  0.274890                   16  0.242348
0.120136       17  0.292334                   17  0.262376
0.168516       18  0.308207                   18  0.282813
0.236378       19  0.322753                   19  0.303431
0.331549       20  0.337128                   20  0.324351
0.465100       21  0.350922                   21  0.342823
0.652400       22  0.312022                   22  0.318798
0.915100       23  0.214893                   23  0.243780
1.283650       24  0.128674                   24  0.165635
1.800600       25  0.072301                   25  0.102046
2.525700       26  0.037997                   26  0.053451
3.542800       27  0.017882                   27  0.019916
4.969550       28  0.009271                   28  0.003579
6.970850       29  0.008300                   29  0.001673
9.778100       30  0.008316                   30  0.004321
13.715850      31  0.007190                   31  0.006000
19.239350      32  0.005119                   32  0.005780
26.987250      33  0.004595                   33  0.006268
37.855300      34  0.005899                   34  0.007922
53.100050      35  0.006596                   35  0.008834
73.887500      36  0.006705                   36  0.009220
97.662500      37  0.006311                   37  0.009007
121.437500     38  0.004633                   38  0.007193
145.212500     39  0.002243                   39  0.004256
168.987500     40  0.000020                   40  0.001229
192.762500     41 -0.002266                   41 -0.001931
216.537500     42 -0.004752                   42 -0.005326
240.312500     43 -0.007623                   43 -0.009093
264.087500     44 -0.011267                   44 -0.013597
287.862500     45 -0.016020                   45 -0.018810
311.637500     46 -0.018440                   46 -0.021263
335.412500     47 -0.016199                   47 -0.018954
359.187500     48 -0.012583                   48 -0.014915
382.962500     49 -0.009519                   49 -0.011291
406.737500     50 -0.006634                   50 -0.007969
430.512500     51 -0.003731                   51 -0.004773
454.287500     52 -0.000727                   52 -0.001632
478.062500     53  0.002415                   53  0.001670
501.837500     54  0.005704                   54  0.005031
525.612500     55  0.009126                   55  0.008534
549.387500     56  0.012639                   56  0.012145
573.162500     57  0.016182                   57  0.015804
596.937500     58  0.019674                   58  0.019419
620.712500     59  0.023028                   59  0.022830
644.487500     60  0.026174                   60  0.026181
668.262500     61  0.029077                   61  0.029248
692.037500     62  0.031750                   62  0.032098
715.812500     63  0.034248                   63  0.034807
739.587500     64  0.036672                   64  0.037485
763.362500     65  0.039147                   65  0.040259
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -7.707148  0.000000 -7.707148 -7.723848  9.403484e-08   
0.000624    2     -7.707162  0.000006 -7.707157 -7.723855  3.799168e-06   
0.000876    3     -7.707168  0.000008 -7.707160 -7.723857  5.378318e-06   
0.001229    4     -7.707176  0.000012 -7.707164 -7.723861  7.846912e-06   
0.001723    5     -7.707186  0.000018 -7.707169 -7.723866  1.169579e-05   
0.002417    6     -7.707199  0.000027 -7.707173 -7.723872  1.769265e-05   
0.003391    7     -7.707217  0.000041 -7.707176 -7.723880  2.702570e-05   
0.004757    8     -7.707239  0.000062 -7.707177 -7.723889  4.152417e-05   
0.006672    9     -7.707267  0.000096 -7.707171 -7.723900  6.402696e-05   
0.009359    10    -7.707301  0.000148 -7.707154 -7.723913  9.890362e-05   
0.013128    11    -7.707342  0.000227 -7.707115 -7.723925  1.528780e-04   
0.018415    12    -7.707387  0.000349 -7.707039 -7.723935  2.362762e-04   
0.025830    13    -7.707434  0.000534 -7.706900 -7.723936  3.649104e-04   
0.036232    14    -7.707474  0.000816 -7.706657 -7.723920  5.628440e-04   
0.050823    15    -7.707495  0.001237 -7.706258 -7.723874  8.633661e-04   
0.071291    16    -7.707486  0.001849 -7.705637 -7.723780  1.310040e-03   
0.100000    17    -7.707425  0.002722 -7.704703 -7.723612  1.966435e-03   
0.140271    18    -7.707267  0.003959 -7.703308 -7.723322  2.927408e-03   
0.196760    19    -7.706936  0.005690 -7.701246 -7.722829  4.327677e-03   
0.275997    20    -7.706298  0.008082 -7.698216 -7.722008  6.354461e-03   
0.387100    21    -7.705115  0.011336 -7.693779 -7.720646  9.262046e-03   
0.543100    22    -7.702953  0.015658 -7.687295 -7.718413  1.336502e-02   
0.761700    23    -7.699786  0.020571 -7.679215 -7.715306  1.851378e-02   
1.068500    24    -7.696282  0.024877 -7.671405 -7.711835  2.390300e-02   
1.498800    25    -7.692813  0.027967 -7.664846 -7.708301  2.881252e-02   
2.102400    26    -7.689476  0.029800 -7.659677 -7.704817  3.262650e-02   
2.949000    27    -7.686411  0.030545 -7.655866 -7.701559  3.472906e-02   
4.136600    28    -7.683910  0.030559 -7.653350 -7.698829  3.480106e-02   
5.802500    29    -7.682121  0.030600 -7.651521 -7.696782  3.345971e-02   
8.139200    30    -7.680823  0.031600 -7.649223 -7.695198  3.233954e-02   
11.417000   31    -7.680009  0.034015 -7.645994 -7.694074  3.289345e-02   
16.014700   32    -7.680214  0.038136 -7.642078 -7.693977  3.606480e-02   
22.464000   33    -7.682756  0.044589 -7.638167 -7.696251  4.275442e-02   
31.510500   34    -7.688672  0.055429 -7.633243 -7.701917  5.513818e-02   
44.200100   35    -7.698112  0.073736 -7.624376 -7.711140  7.627088e-02   
62.000000   36    -7.712372  0.101903 -7.610468 -7.725228  1.089873e-01   
85.775000   37    -7.732667  0.141081 -7.591586 -7.745329  1.550582e-01   
109.550000  38    -7.754238  0.180426 -7.573812 -7.766575  2.016727e-01   
133.325000  39    -7.778409  0.217644 -7.560765 -7.790365  2.457230e-01   
157.100000  40    -7.806731  0.252283 -7.554448 -7.818283  2.856279e-01   
180.875000  41    -7.839611  0.285218 -7.554393 -7.850763  3.215706e-01   
204.650000  42    -7.877560  0.316785 -7.560776 -7.888477  3.538458e-01   
228.425000  43    -7.921340  0.347179 -7.574160 -7.932594  3.829634e-01   
252.200000  44    -7.972174  0.376545 -7.595630 -7.984731  4.094887e-01   
275.975000  45    -8.032385  0.405022 -7.627363 -8.047600  4.340606e-01   
299.750000  46    -8.106042  0.433561 -7.672481 -8.124540  4.580198e-01   
323.525000  47    -8.192537  0.468122 -7.724415 -8.213628  4.872186e-01   
347.300000  48    -8.285319  0.515281 -7.770038 -8.308415  5.286187e-01   
371.075000  49    -8.380328  0.574852 -7.805476 -8.404986  5.831798e-01   
394.850000  50    -8.476627  0.644340 -7.832286 -8.502341  6.487340e-01   
418.625000  51    -8.573539  0.722568 -7.850971 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')